In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
from analysis.lib.pq import pq_tools;reload(pq_tools)

%matplotlib inline
from analysis.lib.fitting import fit, common

folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\TPQI_with_Pippin_SIL3'
folder_lt4_disting = r'D:\measuring\data\Purification_lt4_raw_data\TPQI_final_final_disting'

filename_contains = 'TPQI'

from analysis.lib.purification import purify_TPQI as TPQI; reload(TPQI)

In [ ]:
### define parameters
start_ch0 = 2435e3#+500e3
start_offset_ch1 = 0e3
window_length = 50e3
pulse_sep = 700e3
number_of_pulses = 10

In [ ]:
### get coincidences
# execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
reload(TPQI)
reload(pq_tools)
dts, filtered_dts = TPQI.TPQI_analysis(folder_lt4,start_ch0,start_ch0+start_offset_ch1,
                                       window_length,
                                       pulse_sep,
                                       number_of_pulses,
                                       contains =  filename_contains,
                                       Verbose = False)
len(dts),len(filtered_dts)


# dts_disting, filtered_dts_disting = TPQI.TPQI_analysis(folder_lt4_disting,start_ch0,start_ch0+start_offset_ch1,
#                                                        window_length,
#                                                        pulse_sep,
#                                                        number_of_pulses, 
#                                                        Verbose = False)
len(dts),len(filtered_dts)

In [ ]:
### plot result

lim = 1500

hist_binsize = 5
bins = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,hist_binsize)
y,x=np.histogram(filtered_dts, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(bin_centres,y)
ax.set_xlabel('Difference time (ns)')
ax.set_ylabel('Coincidences in bin')
# ax.set_range
ax.set_title('TPQI')
ax.set_xlim([-lim,lim])
# ax.set_ylim([0,30])

# envelope = lambda t: (75/50) * 5* np.exp(-abs(t)**2/20**2.)*(1-np.cos(2*np.pi*0.040*t))/2
# #envelope = lambda t: (75/50) * 5* np.exp(-abs(t)**2/20**2.)

# xp = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,1)
# env = envelope(xp)
# ax.plot(xp,env)

plt.show()
plt.close('all')


# ### plot result

# bins = np.arange(min(filtered_dts_disting)-100.5,max(filtered_dts_disting)+100.5,hist_binsize)
# y,x=np.histogram(filtered_dts_disting, bins=bins)
# bin_centres = x[:-1] + (x[1] - x[0])/2
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.plot(bin_centres,y)
# ax.set_xlabel('Difference time (ns)')
# ax.set_ylabel('Coincidences in bin')
# # ax.set_range
# ax.set_title('TPQI')
# ax.set_xlim([-lim,lim])
# # ax.set_ylim([0,30])

# envelope_dist = lambda t: 10* np.exp(-abs(t)**2/20**2.)
# xp = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,1)
# env = envelope_dist(xp)
# ax.plot(xp,env)
# plt.show()
# plt.close('all')



In [ ]:
def fit_tpqi(g_a,n_pulses):
    """
    fitfunction for TPQI
        y(x) = a * (n_pulses - x)/n_pulses

    I.g.:
        g_a : amplitude
    """
    fitfunc_str = 'a'

    a = fit.Parameter(g_a, 'a')
    n_pulses = fit.Parameter(n_pulses, 'n_pulses')
    p0 = [a, n_pulses]

    def fitfunc(x):
        return a() * (n_pulses() - x)/n_pulses()

    return p0, fitfunc, fitfunc_str


counts = np.zeros(number_of_pulses)
diff =  range(1,number_of_pulses+1)
for z in diff:
    c_left = np.sum((filtered_dts<(200-z*pulse_sep/1e3)) & (filtered_dts>(-200-z*pulse_sep/1e3)))
    c_right = np.sum((filtered_dts<(200+z*pulse_sep/1e3)) & (filtered_dts>(-200+z*pulse_sep/1e3)))
    counts[z-1] = (c_left + c_right)/2
    
reload(fit)
fitted = fit.fit1d(diff,counts,fit_tpqi,300,10,ret=True)

plt.plot(diff, counts ,'o')
plt.plot(diff,fitted['fitfunc'](diff))
plt.xlabel('Pulse difference')
plt.ylabel('Coincidences')

fitted_counts_in_next_pulse = fitted['params'][0]
print 'Fitted coincidences with next pulse', fitted_counts_in_next_pulse

In [ ]:
counts_in_next_pulse = (np.sum((filtered_dts<-pulse_sep/1e3+200) & (filtered_dts>-pulse_sep/1e3-200))+np.sum((filtered_dts>pulse_sep/1e3-200) & (filtered_dts<pulse_sep/1e3+200)))/2
counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))
print counts_in_next_pulse

print 'counts left',np.sum((filtered_dts<-pulse_sep/1e3+200) & (filtered_dts>-pulse_sep/1e3-200))
print 'counts right',np.sum((filtered_dts>pulse_sep/1e3-200) & (filtered_dts<pulse_sep/1e3+200))
print 'averaged coincidences with following pulses',counts_in_next_pulse 
print 'coincidences in same pulse',counts_in_same_pulse

print fitted_counts_in_next_pulse
tail_lt3 = 7.2 #9.1975 for SIL 3
tail_lt4 = 5.1514

p1 = tail_lt3
p2 = tail_lt4

Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
# Visibility = 0.5
print 'assumed tail LT3/LT4',tail_lt3,tail_lt4
print 'distiungishable expected vis', Visibility
print 'measured Visibility', counts_in_same_pulse/float(counts_in_next_pulse)
print 'measured Visibility', counts_in_same_pulse/float(fitted_counts_in_next_pulse)
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(fitted_counts_in_next_pulse))/Visibility


In [ ]:
def sweep_tpqi_param(param_name,sweep_pts,contains = 'TPQI'):
    vis = []
    vis_u = []
    initialVal = tpqi_dict[param_name]
    for p in sweep_pts:
        tpqi_dict[param_name] = p
        # get dts
        dts, filtered_dts = TPQI.TPQI_analysis(folder_lt4,
                                               tpqi_dict['start_ch0'],tpqi_dict['start_ch0']+tpqi_dict['start_offset_ch1'],
                                               tpqi_dict['window_length'],
                                              tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'], contains = contains,
                                               Verbose = False)
        ## calc visibility
        counts_in_next_pulse = float((np.sum((filtered_dts<-pulse_sep-200) & \
                                       (filtered_dts>-pulse_sep+200))+np.sum((filtered_dts>pulse_sep-200) & \
                                       (filtered_dts<pulse_sep+200)))/2.)
        counts_in_same_pulse = float(np.sum((filtered_dts<200) & (filtered_dts>-200)))

        tail_lt3 = 7.2 #9.1975 for SIL 2 of pippin
        tail_lt4 = 5.1514

        p1 = tail_lt3
        p2 = tail_lt4
        
        counts = np.zeros(number_of_pulses)
        diff =  range(1,number_of_pulses+1)
        for z in diff:
            c_left = np.sum((filtered_dts<(200-z*pulse_sep/1e3)) & (filtered_dts>(-200-z*pulse_sep/1e3)))
            c_right = np.sum((filtered_dts<(200+z*pulse_sep/1e3)) & (filtered_dts>(-200+z*pulse_sep/1e3)))
            counts[z-1] = (c_left + c_right)/2
        fitted = fit.fit1d(diff,counts,fit_tpqi,300,10,ret=True,do_print = False)
        counts_in_next_pulse = fitted['params'][0]
#         print counts_in_next_pulse

        Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
        if counts_in_same_pulse != 0 and counts_in_next_pulse != 0:
            vis.append(1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility)
            vis_u.append(np.sqrt((counts_in_same_pulse**2)/counts_in_next_pulse**3+ \
                                 (counts_in_same_pulse)/counts_in_next_pulse**2)/(Visibility))
#             print counts_in_same_pulse, counts_in_next_pulse,vis_u[-1]
        else:
            vis.append(0)
            vis_u.append(0)
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.errorbar(sweep_pts,vis,vis_u)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Visibility')
    plt.show()
    plt.close('all')
    
    tpqi_dict[param_name] = initialVal
# Error propagation for N1/N2     Sqrt[N2^2 \[Sigma]1^2 + N1^2 \[Sigma]2^2]/N2^2

In [ ]:
##### sweep tpqi parameters
tpqi_dict = {
    'start_ch0' : 2435e3,
    'start_offset_ch1' : 1e3,
    'window_length' : 50e3,
    'pulse_sep' : 700e3,
    'number_of_pulses' : 10,
    'cr_after_threshold' : 0} # to be programmed
sweep_tpqi_param('start_ch0',np.linspace(2415e3,2425e3,15),contains = filename_contains)
sweep_tpqi_param('window_length',np.linspace(10e3,60e3,15),contains = filename_contains)

In [ ]:
### regular histgram plotting:
reload(pq_plots)
# f = tb.latest_data('105117',folder = folder_lt4)
# a = pqsequence.PQSequenceAnalysis(f)
f = tb.latest_data('Bell',folder = folder_lt4)
b = pqsequence.PQSequenceAnalysis(f)

# pq_plots.plot_photon_hist(a.pqf,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log = False)
pq_plots.plot_photon_hist(b.pqf,hist_binsize = 1e2,start=2.410e6,length=0.040e6,log = False)
# a.plot_histogram(0,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)
# a.plot_histogram(1,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)

In [ ]:
LDE_attemps_per_rep = 100


f = tb.latest_data('150410',folder = folder_lt4)
a = pqsequence.PQSequenceAnalysis(f)
a.reps = 50000 # i got this from the hdf5 file
# a.plot_histogram(0,hist_binsize = 1e3,start=2.410e6+9*.7e6,length=0.05e6)
a.plot_histogram(1,hist_binsize = 5e2,start=2.43e6,length=0.01e6,log_plot = False)
(h0, b0), (h1, b1) = pq_tools.get_photon_hist(a.pqf, index = 1, start = 2.434e6,length = 0.0025e6,hist_binsize = 2e2)
h_ZPL = h1
h_ZPL = np.append(h_ZPL, h_ZPL[-1])#/(a.reps*LDE_attemps_per_rep)

f = tb.latest_data('170029',folder = r'D:\measuring\data\Purification_lt4_raw_data\TPQI_3_laserPulse')
a = pqsequence.PQSequenceAnalysis(f)
a.reps = 50000
(h0, b0), (h1, b1) = pq_tools.get_photon_hist(a.pqf, index = 1, start = 2.434e6,length = 0.0025e6,hist_binsize = 2e2)
h_laser = h1
h_laser = np.append(h_laser, h_laser[-1])#/(a.reps*LDE_attemps_per_rep)

fig = plt.figure()
ax = plt.subplot()
plt.plot(b0,h_ZPL)
plt.plot(b0+0.5e3,h_laser*10)
ax.set_yscale('log')
ax.set_ylim(0.1,2000)
ax.set_xlim(2.433e6,2.436e6)
plt.show()
# a.plot_histogram(0,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)
# a.plot_histogram(1,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)



In [ ]:
### fitting the laser pulses:
(h0, b0), (h1, b1) = pq_tools.get_photon_hist(a.pqf, index = 1, start = 2.430e6,length = 0.01e6,hist_binsize = 1e2)
channel = 0
#fit laser pulse data
ff = common.fit_gauss
y_laser = h0 if channel == 0 else h1
y_laser = np.append(y_laser, y_laser[-1])

args=[0.1,3,2433.73e3,2e3]
fitres = fit.fit1d(b0, y_laser, ff, *args, fixed = [0],
                   do_print = True, ret = True)
x0_laser = fitres['params_dict']['x0']
A_laser = fitres['params_dict']['A']
fig = plt.figure()
ax = plt.subplot()
plt.plot(b0,y_laser)
plot.plot_fit1d(fitres, np.linspace(2.430e6,2.430e6+0.01e6,201),ax=ax, plot_data=False,add_txt=False, lw = 2)



channel = 1
#fit laser pulse data
ff = common.fit_gauss
y_laser = h0 if channel == 0 else h1
y_laser = np.append(y_laser, y_laser[-1])

args=[0.1,3,2433.73e3,2e3]
fitres = fit.fit1d(b0, y_laser, ff, *args, fixed = [0],
                   do_print = True, ret = True)
x0_laser = fitres['params_dict']['x0']
A_laser = fitres['params_dict']['A']
plt.plot(b0,y_laser)
plot.plot_fit1d(fitres, np.linspace(2.430e6,2.430e6+0.01e6,201),ax=ax, plot_data=False,add_txt=False, lw = 2)


In [ ]:
#### lets have a look at the normalization measurements:
### LT4
folder_normalization = r'D:\measuring\data\Purification_lt4_raw_data\TPQI_2_normalization'
f = tb.latest_data('norm_LT4',folder = folder_normalization)
a = pqsequence.PQSequenceAnalysis(f)
a.reps = 10
a.plot_histogram(0,hist_binsize = 1e3,start=2.418e6,length=0.05e6)
# a.g.attrs['completed_reps']
### LT3
f = tb.latest_data('norm_LT3',folder = folder_normalization)
a = pqsequence.PQSequenceAnalysis(f)
a.reps = 10
a.plot_histogram(0,hist_binsize = 1e3,start=2.418e6,length=0.05e6)
# a.g.attrs['completed_reps']

# aNALYSIS OF THE lde RUNS

In [ ]:
folder_TPQI_LDE = r'D:\measuring\data\Purification_lt4_raw_data\TPQI_from_LDE_2'
from analysis.lib.bell.old import TPQI; reload(TPQI)

In [ ]:
### define parameters
start_ch0 = 2.500e6#+500e3
start_offset_ch1 = 0#1.5e3
window_length = 50e3
pulse_sep = 500e3
number_of_pulses = 2

In [ ]:
### get coincidences
reload(TPQI)
dts, filtered_dts = TPQI.TPQI_analysis(folder_TPQI_LDE,start_ch0,start_ch0+start_offset_ch1,window_length,pulse_sep,number_of_pulses)

In [ ]:
### plot result
hist_binsize = 15
bins = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,hist_binsize)
y,x=np.histogram(filtered_dts, bins=bins)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x[:-1],y)
ax.set_xlabel('Difference time (ns)')
ax.set_ylabel('Coincidences in bin')
# ax.set_range
ax.set_title('TPQI')
# ax.set_xlim([-800,800])
plt.show()
plt.close('all')

### check pulse position

In [ ]:
#### lets have a look at the normalization measurements:
### LT4

f = tb.latest_data('XX',folder = folder_TPQI_LDE)
a = pqsequence.PQSequenceAnalysis(f)
a.reps = 10
a.plot_histogram(0,hist_binsize = 1e3,start=2.473e6+0.5e6,length=0.09e6)
# a.g.attrs['completed_reps']

In [8]:
import numpy as np
nf_start = 0
nf_sweep = 0.5
scan_min = nf_start-nf_sweep
scan_max = nf_start +nf_sweep
opt_nf_step = 0.1
steps=int((scan_max - scan_min) / opt_nf_step)
nf_sweep=np.append(np.linspace(nf_start,scan_min+opt_nf_step,int(steps/2.)),
        np.linspace(scan_min, scan_max, steps))
nf_sweep=np.append(nf_sweep,np.linspace(scan_max-opt_nf_step,nf_start,int(steps/2.)))       

In [10]:
nf_sweep[::-1]

array([ 0.        ,  0.1       ,  0.2       ,  0.3       ,  0.4       ,
        0.5       ,  0.38888889,  0.27777778,  0.16666667,  0.05555556,
       -0.05555556, -0.16666667, -0.27777778, -0.38888889, -0.5       ,
       -0.4       , -0.3       , -0.2       , -0.1       ,  0.        ])